# Initialize

In [1]:
from azureml.core import Workspace
import json

ws = Workspace.from_config('./.config/azureml_workspace.json')
with open("./.config/auth_keys.json") as auth_keys_file:
    auth_keys = json.load(auth_keys_file)

# Prepare data

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import shutil

from azureml.core import Dataset

datasets.CIFAR10(root='./data', train=True, download=True)
datasets.CIFAR10(root='./data', train=False, download=True)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='datasets/cifar10', overwrite=True)
Dataset.File.from_files(path=(datastore, 'datasets/cifar10')).register(workspace=ws, name='CIFAR10')

# Explore dataset

In [14]:
from torch.utils.data import DataLoader
import torch
from torchvision import datasets, transforms

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
loader = DataLoader(dataset, batch_size=10, shuffle=False)
mean, std = 0, 0
for images, _ in loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)

mean /= len(dataset)
std /= len(dataset)

print(mean)
print(std)

Files already downloaded and verified
tensor([0.4914, 0.4822, 0.4465])
tensor([0.2023, 0.1994, 0.2010])


# Configure environment

In [3]:
from azureml.core import Environment

env = Environment.from_conda_specification('PyTorch-1.8.1-GPU', 'environment.yml')
env.docker.base_image = ('mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04')
env.register(ws);

# Submit run

In [5]:
from azureml.core import Experiment, Dataset, Environment, ComputeTarget
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

experiment_name = "CIFAR10"

env = Environment.get(workspace=ws, name='PyTorch-1.8.1-GPU')
env.environment_variables['WANDB_API_KEY'] = auth_keys['wandb']
env.environment_variables['WANDB_ENTITY'] = 'skovacevic94'
env.environment_variables['WANDB_PROJECT'] = experiment_name

script_config = ScriptRunConfig(
    source_directory="./src",
    script='main.py',
    arguments=[
            '--data_path', Dataset.get_by_name(workspace=ws, name='CIFAR10').as_mount(),
            '--lr', 0.001,
            '--batch_size', 128,
            '--max_epoch', 100,
            '--seed', 42],
    environment=env,
    compute_target=ComputeTarget(workspace=ws, name='BasicK80')
)

run = Experiment(workspace=ws, name=experiment_name).submit(config=script_config)
RunDetails(run).show()

run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "'NeuralNetwork' object has no attribute 'conv4'",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"'NeuralNetwork' object has no attribute 'conv4'\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}